In [ ]:
import numpy as np
import plotly.graph_objects as go

# Пространственная и временная сетка
L = 1.0
h = 0.1
tau = 0.001
T = 0.05

x = np.arange(0, L + h, h)
y = np.arange(0, L + h, h)
X, Y = np.meshgrid(x, y)

Nx, Ny = len(x), len(y)
Nt = int(T / tau)

# Точное решение
def u_exact(t, x, y):
    return t * np.sin(np.pi * x) * np.sin(np.pi * y)

# Правая часть
def f_rhs(t, x, y):
    return np.sin(np.pi * x) * np.sin(np.pi * y) * (1 + 2 * np.pi**2 * t)

# Инициализация
v = np.zeros((Nx, Ny))
v_half = np.zeros_like(v)
v_new = np.zeros_like(v)
frames = []

# Лапласиан
def laplacian(u, h):
    lap = np.zeros_like(u)
    lap[1:-1,1:-1] = (
        u[2:,1:-1] + u[:-2,1:-1] + u[1:-1,2:] + u[1:-1,:-2] - 4 * u[1:-1,1:-1]
    ) / h**2
    return lap

# Временной цикл с сохранением кадров
save_interval = 5  # каждый 5-й шаг сохраняем
for n in range(Nt):
    t_n = n * tau
    t_half = t_n + 0.5 * tau

    # Полушаг
    lap_v = laplacian(v, h)
    v_half[1:-1,1:-1] = v[1:-1,1:-1] + tau * (
        lap_v[1:-1,1:-1] + f_rhs(t_n, X[1:-1,1:-1], Y[1:-1,1:-1])
    )
    v_half[0,:] = v_half[-1,:] = v_half[:,0] = v_half[:,-1] = 0

    # Полный шаг
    lap_v_half = laplacian(v_half, h)
    v_new[1:-1,1:-1] = v_half[1:-1,1:-1] + tau * (
        lap_v_half[1:-1,1:-1] + f_rhs(t_half, X[1:-1,1:-1], Y[1:-1,1:-1])
    )
    v_new[0,:] = v_new[-1,:] = v_new[:,0] = v_new[:,-1] = 0
    v = v_new.copy()

    if n % save_interval == 0:
        frames.append(go.Frame(
            data=[go.Surface(z=v.copy(), x=X, y=Y, colorscale='Viridis')],
            name=f'{n * tau:.4f}'
        ))

# Базовая фигура
fig = go.Figure(
    data=[go.Surface(z=frames[0].data[0].z, x=X, y=Y, colorscale='Viridis')],
    layout=go.Layout(
        title='Эволюция температуры во времени',
        scene=dict(zaxis=dict(range=[0, T])),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="▶", method="animate", args=[None])]
        )]
    ),
    frames=frames
)

fig.show()
